In [ ]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import undetected_chromedriver as uc
import locale

import calendar
import random

In [ ]:
# json_pages_info = {"pages": []}

In [ ]:
import pyautogui
import random

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

json_pages_info = {"pages": []}

pages = 10

re = False

for page in range(0, pages):

	first = True
	while (re or first):
		first = False
		
		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Awww.theclinic.cl&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'
				driver.get(url)

			re = False
			time.sleep(random.uniform(6, 12))

			articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
			articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

			for article in articles:
				print("revisando artículo...")
				try: 
					originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
					for es, en in months.items():
						if es in originalDate:
								originalDate = originalDate.replace(es, en)
								break
						
					date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

					if (date_epoch < 1573786800 or date_epoch > 1702782000):
						print("La noticia no corresponde a la fecha solicitada")
						continue

					# check if the article is from El Mostrador
					if (not ("www.theclinic.cl" in article.text)):
						print("El artículo no es de The Clinic")
						continue

					try:
						title = article.find_element(By.TAG_NAME, "h3").text
						description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
						link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

						link_info = {
							"newscast" : "The Clinic",
							"title": title,
							"description": description,
							"category": "The site does not provide a category",
							"date": originalDate,
							"image_link": "not found initially",
							"author": "not found initially",
							"link": link,
						}

						print("información incluida!")
						json_pages_info["pages"].append(link_info)

					except Exception as e:
						print(f"Error al extraer información del artículo: {e}")
						continue

				except Exception as e:
					print(f"Error al procesar el artículo: {e}")
					continue
		except Exception as e:
			print(f"Error al procesar el resultado de busqueda. Página: {page}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/the_clinic.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


# with open(f"output.html", "w", encoding="utf-8") as f:
# 	f.write(driver.page_source)

# Content Extraction

In [24]:
extracted_pages = {"pages": []}
extracted_pages_with_content = {"pages": []}
last_extracted_pages_with_content = {"pages": []}

total_pages = 0
pages_succeeded = 0
pages_failed = 0
error_occurred = False
driver = None
ad_message = "Suscríbase aquí a la newsletter de EL PAÍS América y reciba todas las claves informativas de la actualidad de la región."
limit_of_pages = "all"
start_page = 0

content = ""
author = ""

test_specific_url = ""

def recursive_search(element):
		element_tag_name = element.tag_name
		# get class of a element
		if element.get_attribute("class") == "bloque-tc-dos-recomendados":
				return ""
		if element_tag_name == "p" or element_tag_name == "span" or element_tag_name == "h2" or element_tag_name == "h3" or element_tag_name == "h4" or element_tag_name == "a":
				return element.text.strip()
		
		children = element.find_elements(By.XPATH, ".//*")
		for child in children:
				result = recursive_search(child)
				if result:
						return result
		return ""

print("[INFO] Loading extracted pages from JSON file...")
with open("../archive/pages_extracted/the_clinic/the_clinic_240725.json", 'r', encoding='utf-8') as file:
		extracted_pages = json.load(file)
		print(f"[INFO] {len(extracted_pages['pages'])} pages loaded.")
		if limit_of_pages == "all":
				limit_of_pages = len(extracted_pages["pages"])
				print(f"[INFO] Limit of pages set to {limit_of_pages}")

# with open("../archive/temp/pages_with_content/the_clinic_240725_with_content_reference.json", 'r', encoding='utf-8') as file:
# 		last_extracted_pages_with_content = json.load(file)
# 		print(f"[INFO] {len(last_extracted_pages_with_content['pages'])} pages loaded.")


if start_page >= len(extracted_pages["pages"]):
		print(f"[ERROR] start_page is out of range. The maximum page index is {len(extracted_pages['pages']) - 1}.")
		raise SystemExit

def extract_from_DOM():
		print("[INFO] Trying to extract content from DOM...")
		content = ""
		author = ""

		content_container = body.find_element(By.CLASS_NAME, "the-content")
		# paragraph = content_container.find_elements(By.TAG_NAME, "p") if content_container else None
		paragraph_elements = content_container.find_elements(By.XPATH, "./*")
		content = ""

		for element in paragraph_elements:
			p = recursive_search(element)
			if "Volver al Home" in p or "También puedes leer" in p:
					continue
			content += p + "\n"

		try:
				author = body.find_element(By.CLASS_NAME, "autor").find_element(By.TAG_NAME, "a").text.strip()
		except:
				author = "not found"

		print("[INFO] The content has been extracted. Method: DOM", flush=True)
		if len(content) < 100:
				print("[WARNING] The extracted content is too short.", flush=True)
				print(f"[WARNING] Content: {content}", flush=True)
		return content, author

data_extracted = True
re = False

print("[INFO] Starting extraction process...")
for page_index, page in enumerate(extracted_pages["pages"]):

		if test_specific_url == "":
			if total_pages >= limit_of_pages:
					print("[INFO] Limit of pages reached.")
					break

			if "content" not in page.keys():
					page["content"] = "The content has not been extracted yet"

			if page_index < start_page:
					if page_index < len(last_extracted_pages_with_content["pages"]):
							extracted_pages_with_content["pages"].append(last_extracted_pages_with_content["pages"][page_index])
					else:
						extracted_pages_with_content["pages"].append(page)
					print("[INFO] Skipping page: ", page_index)
					continue
			if page["content"] != "The content has not been extracted yet":
					print(f"[INFO] Page {page_index} already has content. Skipping...")
					extracted_pages_with_content["pages"].append(page)
					continue
		else:
			print("[INFO] Processing specific page: ", test_specific_url)
			for page_aux in extracted_pages["pages"]:
				if page_aux["link"] == test_specific_url:
					page = page_aux
					test_specific_url = ""
					total_pages = 99999
					print("[INFO] The specific page found!")
					break

		total_pages += 1

		print(f"\n[INFO] Checking page: {page_index} | Link: {page['link']}" )
		url = page['link']
		first = True

		while (re or first):
			first = False
			
			try: 
				time.sleep(1)

				if (not re): 
					options = uc.ChromeOptions()
					options.add_argument("--no-sandbox")
					options.add_argument("--disable-blink-features=AutomationControlled")

					driver = uc.Chrome(options=options)
					print("[INFO] Chrome driver initialized.")
					driver.get(url)
					print("[INFO] Page loaded.")

				re = False
				time.sleep(random.uniform(6, 12))
				body = driver.find_element(By.TAG_NAME, 'body')

				try: 
					content, author = extract_from_DOM()

					# Clean up ad message
					content = content.replace(ad_message, "").strip()
				except Exception as e:
					print(f"[ERROR] A error occurred while extracting content", flush=True)
					raise

			except Exception as e:
				print(f"[ERROR] A error occurred while processing the page: {page_index}", flush=True)
				print(f"[ERROR] Site: {url}", flush=True)
				print(e, flush=True)
				response = input("Retry? y/n")
				if response.lower() == 'y':
					print("[INFO] Retry...")
					re = True
				else: 
					response = input("Description? Press enter to not add a description")
					if not response:
							response = "No description provided"
					else: 
							print(f"[INFO] The user add a description: {response}")
					print("[INFO] Cancelling...")
					error_occurred = True
					page["Observation"] = "A error occurred while extracting content. The user cancelled the operation. Description: " + response
					re = False
			
			if (not re):
				driver.quit()

		if error_occurred:
			content = "A error occurred while extracting content"
			author = "A error occurred while extracting content"
			pages_failed += 1
			error_occurred = False
		else:
			pages_succeeded += 1

		page["content"] = content
		page["author"] = author
		extracted_pages_with_content["pages"].append(page)

		print("[INFO] Storing information in a JSON file...")
		with open(f"../archive/temp/pages_with_content/the_clinic_240725_with_content.json", 'w', encoding='utf-8') as file:
			json.dump(extracted_pages_with_content, file, ensure_ascii=False, indent=4)

		driver.quit()

		time.sleep(random.uniform(6, 12))

# ----------------- Stats -----------------
print("\n\n")
print("----- STATS -----")

try:
	print(f"Amount of pages processed: {total_pages}")
	print(f"% Pages succeeded [{pages_succeeded}]: {round(100 * pages_succeeded/total_pages, 2)}")
	print(f"% Pages failed [{pages_failed}]: {round(100 * pages_failed/total_pages, 2)}")
except: 
	print("A error occurred while processing the stats")

[INFO] Loading extracted pages from JSON file...
[INFO] 89 pages loaded.
[INFO] Limit of pages set to 89
[INFO] Starting extraction process...

[INFO] Checking page: 0 | Link: https://www.theclinic.cl/2021/10/18/18-o-en-que-estan-siete-rostros-clave-del-estallido-social/
[INFO] Chrome driver initialized.
[INFO] Page loaded.
[INFO] Trying to extract content from DOM...
[INFO] The content has been extracted. Method: DOM
[INFO] Storing information in a JSON file...

[INFO] Checking page: 1 | Link: https://www.theclinic.cl/2020/10/01/la-primera-chispa-a-un-ano-de-la-polemica-alza-del-pasaje-que-detono-el-estallido-social/
[INFO] Chrome driver initialized.
[INFO] Page loaded.
[INFO] Trying to extract content from DOM...
[INFO] The content has been extracted. Method: DOM
[INFO] Storing information in a JSON file...

[INFO] Checking page: 2 | Link: https://www.theclinic.cl/2023/10/09/huellas-del-estallido-social-imagenes-zona-cero-a-cuatro-anos/
[INFO] Chrome driver initialized.
[INFO] Page l